In [ ]:
!pip install wandb 

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import  Conv2D,MaxPool2D,Flatten,Dense

import wandb
from wandb.keras import WandbCallback

import numpy as np
import cv2

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
wandb.init(project="Sheikh Recognizer")

## Dataset

In [ ]:
dataset_path = "/content/drive/MyDrive/Datasets/Sheykh-Recognition"

width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    rotation_range = 15,
    zoom_range = 0.1,
    brightness_range =  (0.9, 1.1), 
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'training'
)

val_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'validation'
)

## Model

In [14]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights = 'imagenet',
    input_shape = (width, height, 3),
)

In [15]:
for layer in base_model.layers:
  layer.trainable = False

In [16]:
model = tf.keras.Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),   
    Dense(2, activation='softmax')                    
])

In [17]:
config=wandb.config
config.learning_rate=0.001

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(config.learning_rate),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics =['accuracy'] )

## Train

In [ ]:
model.fit(train_data, validation_data= val_data, epochs=10,callbacks=[WandbCallback()])

## Test

In [ ]:
dataset_path = "/content/drive/MyDrive/Datasets/Sheykh Test"

width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255
)

test_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical'
)

In [25]:
model.evaluate(test_data)

2/2 [==============================] - 3s 2s/step - loss: 1.0117 - accuracy: 0.8235


[1.0116586685180664, 0.8235294222831726]

## Save Model

In [20]:
model.save("Sheykh Recognizer.h5")

## Inference

In [23]:
img = cv2.imread("/content/1.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (width, height))
img = img / 255.0
img = img.reshape(1, width, height, 3)

result = model.predict(img)

pred = np.argmax(result)

if pred == 0:
  print("Normal")
elif pred == 1:
  print("Sheykh")

Sheykh
